In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
chicken_df = pd.read_csv('D:/pythonTest/DataShipJo/chicken_data.csv', names=['가게명', '총평점', '주소', '아이디', '평점', '날짜', '리뷰'])
chicken_df.head()

,,가게명,총평점,주소,아이디,평점,날짜,리뷰
1,2,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,de**님,1,2시간 전,잘 먹었습니다
2,3,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,pr**님,4.666666667,4시간 전,양이 많이 줄어든 느낌 이네요.
3,4,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,sk**님,4.666666667,8시간 전,오늘도 맛나게 잘 먹었습니다~~~
4,5,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,gh**님,4.666666667,10시간 전,맛있어요 양도 많아요
5,6,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,ki**님,4.666666667,17시간 전,맛있게 잘먹었어요 다음에 또주문할께요


In [5]:
chicken_df['평점'] = str(chicken_df['평점'])

In [24]:
chicken_df

,,가게명,총평점,주소,아이디,평점,날짜,리뷰
1,2,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,de**님,1,2시간 전,잘 먹었습니다
2,3,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,pr**님,4.666666667,4시간 전,양이 많이 줄어든 느낌 이네요.
3,4,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,sk**님,4.666666667,8시간 전,오늘도 맛나게 잘 먹었습니다~~~
4,5,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,gh**님,4.666666667,10시간 전,맛있어요 양도 많아요
5,6,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,ki**님,4.666666667,17시간 전,맛있게 잘먹었어요 다음에 또주문할께요
...,...,...,...,...,...,...,...,...
107546,105358,후라이드참잘하는집-화곡점,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,qa**님,맛 ★ 5 양 ★ 5 배달 ★ 5,2022년 02월 15일,순삭.....ㅎㅎ 죄송합니다 넘 후딱 먹어서 사진 찍을 여유가...^^ 정말 맛있게...
107547,105359,후라이드참잘하는집-화곡점,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,oi**님,맛 ★ 5 양 ★ 5 배달 ★ 5,2022년 02월 15일,사진을 깜빡하고 먹었어요ㅠㅜ 맛은 맛있습니당
107548,105360,후라이드참잘하는집-화곡점,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,wj**님,맛 ★ 5 양 ★ 5 배달 ★ 5,2022년 02월 15일,맛도짱 서비스도 짱짱 또 시켜먹을게요~ 이벤트 많이 해주세요
107549,105361,후라이드참잘하는집-화곡점,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,dk**님,맛 ★ 5 양 ★ 5 배달 ★ 5,2022년 02월 14일,배달도 예정보다 빠르게오고 항상 맛있습니다!!


In [3]:
# 평점의 맛, 양, 배달, ★, 띄워쓰기 삭제

chicken_df["평점"] = chicken_df["평점"].str.replace(pat=r'[★맛양배달 ]', repl=r'', regex=True)

In [4]:
# 평점 값 문자열을 float으로 바꾸기 (노가다 버전)

### 평점 두자릿수 11~55 ###
평점11 = (1+1)/2; 평점12 = (1+2)/2; 평점13 = (1+3)/2; 평점14 = (1+4)/2; 평점15 = (1+5)/2;

평점21 = (2+1)/2; 평점22 = (2+2)/2; 평점23 = (2+3)/2; 평점24 = (2+4)/2; 평점25 = (2+5)/2;

평점31 = (3+1)/2; 평점32 = (3+2)/2; 평점33 = (3+3)/2; 평점34 = (3+4)/2; 평점35 = (3+5)/2;

평점41 = (4+1)/2; 평점42 = (4+2)/2; 평점43 = (4+3)/2; 평점44 = (4+4)/2; 평점45 = (4+5)/2;

평점51 = (5+1)/2; 평점52 = (5+2)/2; 평점53 = (5+3)/2; 평점54 = (5+4)/2; 평점55 = (5+5)/2;


### 평점 111~155 ###

평점111 = (1+1+1)/3; 평점112 = (1+1+2)/3; 평점113 = (1+1+3)/3; 평점114 = (1+1+4)/3; 평점115 = (1+1+5)/3;
평점121 = (1+2+1)/3; 평점122 = (1+2+2)/3 ;평점123 = (1+2+3)/3 ;평점124 = (1+2+4)/3 ;평점125 = (1+2+5)/3;
평점131 = (1+3+1)/3 ;평점132 = (1+3+2)/3 ;평점133 = (1+3+3)/3 ;평점134 = (1+3+4)/3 ;평점135 = (1+3+5)/3;
평점141 = (1+4+1)/3; 평점142 = (1+4+2)/3; 평점143 = (1+4+3)/3; 평점144 = (1+4+4)/3; 평점145 = (1+4+5)/3;
평점151 = (1+5+1)/3; 평점152 = (1+5+2)/3; 평점153 = (1+5+3)/3; 평점154 = (1+5+4)/3; 평점155 = (1+5+5)/3;

### 평점 211~255 ###

평점211 = (2+1+1)/3; 평점212 = (2+1+2)/3; 평점213 = (2+1+3)/3; 평점214 = (2+1+4)/3; 평점215 = (2+1+5)/3;
평점221 = (2+2+1)/3; 평점222 = (2+2+2)/3; 평점223 = (2+2+3)/3; 평점224 = (2+2+4)/3; 평점225 = (2+2+5)/3;
평점231 = (2+3+1)/3; 평점232 = (2+3+2)/3; 평점233 = (2+3+3)/3; 평점234 = (2+3+4)/3; 평점235 = (2+3+5)/3;
평점241 = (2+4+1)/3; 평점242 = (2+4+2)/3; 평점243 = (2+4+3)/3; 평점244 = (2+4+4)/3; 평점245 = (2+4+5)/3;
평점251 = (2+5+1)/3; 평점252 = (2+5+2)/3; 평점253 = (2+5+3)/3; 평점254 = (2+5+4)/3; 평점255 = (2+5+5)/3;

### 평점 311~355 ###
평점311 = (3+1+1)/3; 평점312 = (3+1+2)/3; 평점313 = (3+1+3)/3; 평점314 = (3+1+4)/3; 평점315 = (3+1+5)/3;
평점321 = (3+2+1)/3; 평점322 = (3+2+2)/3; 평점323 = (3+2+3)/3; 평점324 = (3+2+4)/3; 평점325 = (3+2+5)/3;
평점331 = (3+3+1)/3; 평점332 = (3+3+2)/3; 평점333 = (3+3+3)/3; 평점334 = (3+3+4)/3; 평점335 = (3+3+5)/3;
평점341 = (3+4+1)/3; 평점342 = (3+4+2)/3; 평점343 = (3+4+3)/3; 평점344 = (3+4+4)/3; 평점345 = (3+4+5)/3;
평점351 = (3+5+1)/3; 평점352 = (3+5+2)/3; 평점353 = (3+5+3)/3; 평점354 = (3+5+4)/3; 평점355 = (3+5+5)/3;

### 평점 411~455 ###

평점411 = (4+1+1)/3; 평점412 = (4+1+2)/3; 평점413 = (4+1+3)/3; 평점414 = (4+1+4)/3; 평점415 = (4+1+5)/3;
평점421 = (4+2+1)/3; 평점422 = (4+2+2)/3; 평점423 = (4+2+3)/3; 평점424 = (4+2+4)/3; 평점425 = (4+2+5)/3;
평점431 = (4+3+1)/3; 평점432 = (4+3+2)/3; 평점433 = (4+3+3)/3; 평점434 = (4+3+4)/3; 평점435 = (4+3+5)/3;
평점441 = (4+4+1)/3; 평점442 = (4+4+2)/3; 평점443 = (4+4+3)/3; 평점444 = (4+4+4)/3; 평점445 = (4+4+5)/3;
평점451 = (4+5+1)/3; 평점452 = (4+5+2)/3; 평점453 = (4+5+3)/3; 평점454 = (4+5+4)/3; 평점455 = (4+5+5)/3;

### 평점 511~555 ###
평점511 = (5+1+1)/3; 평점512 = (5+1+2)/3; 평점513 = (5+1+3)/3; 평점514 = (5+1+4)/3; 평점515 = (5+1+5)/3;
평점521 = (5+2+1)/3; 평점522 = (5+2+2)/3; 평점523 = (5+2+3)/3; 평점524 = (5+2+4)/3; 평점525 = (5+2+5)/3;
평점531 = (5+3+1)/3; 평점532 = (5+3+2)/3; 평점533 = (5+3+3)/3; 평점534 = (5+3+4)/3; 평점535 = (5+3+5)/3;
평점541 = (5+4+1)/3; 평점542 = (5+4+2)/3; 평점543 = (5+4+3)/3; 평점544 = (5+4+4)/3; 평점545 = (5+4+5)/3;
평점551 = (5+5+1)/3; 평점552 = (5+5+2)/3; 평점553 = (5+5+3)/3; 평점554 = (5+5+4)/3; 평점555 = (5+5+5)/3;

In [5]:
### 평점 두자릿수 11~55 ###
chicken_df = chicken_df.replace({'평점':'11'},평점11); chicken_df = chicken_df.replace({'평점':'12'},평점12);
chicken_df = chicken_df.replace({'평점':'13'},평점13); chicken_df = chicken_df.replace({'평점':'14'},평점14);
chicken_df = chicken_df.replace({'평점':'15'},평점15);

chicken_df = chicken_df.replace({'평점':'21'},평점11); chicken_df = chicken_df.replace({'평점':'22'},평점11);
chicken_df = chicken_df.replace({'평점':'23'},평점11); chicken_df = chicken_df.replace({'평점':'24'},평점11);
chicken_df = chicken_df.replace({'평점':'25'},평점11);

chicken_df = chicken_df.replace({'평점':'31'},평점11); chicken_df = chicken_df.replace({'평점':'32'},평점11);
chicken_df = chicken_df.replace({'평점':'33'},평점11); chicken_df = chicken_df.replace({'평점':'34'},평점11);
chicken_df = chicken_df.replace({'평점':'35'},평점11);

chicken_df = chicken_df.replace({'평점':'41'},평점11); chicken_df = chicken_df.replace({'평점':'42'},평점11);
chicken_df = chicken_df.replace({'평점':'43'},평점11); chicken_df = chicken_df.replace({'평점':'44'},평점11);
chicken_df = chicken_df.replace({'평점':'45'},평점11);

chicken_df = chicken_df.replace({'평점':'51'},평점11); chicken_df = chicken_df.replace({'평점':'52'},평점11);
chicken_df = chicken_df.replace({'평점':'53'},평점11); chicken_df = chicken_df.replace({'평점':'54'},평점11);
chicken_df = chicken_df.replace({'평점':'55'},평점11);


### 평점 111~155 ###
chicken_df = chicken_df.replace({'평점':'111'},평점111); chicken_df = chicken_df.replace({'평점':'112'},평점112);
chicken_df = chicken_df.replace({'평점':'113'},평점113); chicken_df = chicken_df.replace({'평점':'114'},평점114);
chicken_df = chicken_df.replace({'평점':'115'},평점115);

chicken_df = chicken_df.replace({'평점':'121'},평점121); chicken_df = chicken_df.replace({'평점':'122'},평점122);
chicken_df = chicken_df.replace({'평점':'123'},평점123); chicken_df = chicken_df.replace({'평점':'124'},평점124);
chicken_df = chicken_df.replace({'평점':'125'},평점125);

chicken_df = chicken_df.replace({'평점':'131'},평점131); chicken_df = chicken_df.replace({'평점':'132'},평점132);
chicken_df = chicken_df.replace({'평점':'133'},평점133); chicken_df = chicken_df.replace({'평점':'134'},평점134);
chicken_df = chicken_df.replace({'평점':'135'},평점135);

chicken_df = chicken_df.replace({'평점':'141'},평점141); chicken_df = chicken_df.replace({'평점':'142'},평점142);
chicken_df = chicken_df.replace({'평점':'143'},평점143); chicken_df = chicken_df.replace({'평점':'144'},평점144);
chicken_df = chicken_df.replace({'평점':'145'},평점145);

chicken_df = chicken_df.replace({'평점':'151'},평점151); chicken_df = chicken_df.replace({'평점':'152'},평점152);
chicken_df = chicken_df.replace({'평점':'153'},평점153); chicken_df = chicken_df.replace({'평점':'154'},평점154);
chicken_df = chicken_df.replace({'평점':'155'},평점155);

### 평점 211~255 ###

chicken_df = chicken_df.replace({'평점':'211'},평점211); chicken_df = chicken_df.replace({'평점':'212'},평점212);
chicken_df = chicken_df.replace({'평점':'213'},평점213); chicken_df = chicken_df.replace({'평점':'214'},평점214);
chicken_df = chicken_df.replace({'평점':'215'},평점215);

chicken_df = chicken_df.replace({'평점':'221'},평점221); chicken_df = chicken_df.replace({'평점':'222'},평점222);
chicken_df = chicken_df.replace({'평점':'223'},평점223); chicken_df = chicken_df.replace({'평점':'224'},평점224);
chicken_df = chicken_df.replace({'평점':'225'},평점225);

chicken_df = chicken_df.replace({'평점':'231'},평점231); chicken_df = chicken_df.replace({'평점':'232'},평점232);
chicken_df = chicken_df.replace({'평점':'233'},평점233); chicken_df = chicken_df.replace({'평점':'234'},평점234);
chicken_df = chicken_df.replace({'평점':'235'},평점235);

chicken_df = chicken_df.replace({'평점':'241'},평점241); chicken_df = chicken_df.replace({'평점':'242'},평점242);
chicken_df = chicken_df.replace({'평점':'243'},평점243); chicken_df = chicken_df.replace({'평점':'244'},평점244);
chicken_df = chicken_df.replace({'평점':'245'},평점245);

chicken_df = chicken_df.replace({'평점':'251'},평점251); chicken_df = chicken_df.replace({'평점':'252'},평점252);
chicken_df = chicken_df.replace({'평점':'253'},평점253); chicken_df = chicken_df.replace({'평점':'254'},평점254);
chicken_df = chicken_df.replace({'평점':'255'},평점255);

### 평점 311~355 ###

chicken_df = chicken_df.replace({'평점':'311'},평점311); chicken_df = chicken_df.replace({'평점':'312'},평점312);
chicken_df = chicken_df.replace({'평점':'313'},평점313); chicken_df = chicken_df.replace({'평점':'314'},평점314);
chicken_df = chicken_df.replace({'평점':'315'},평점315);

chicken_df = chicken_df.replace({'평점':'321'},평점321); chicken_df = chicken_df.replace({'평점':'322'},평점322);
chicken_df = chicken_df.replace({'평점':'323'},평점323); chicken_df = chicken_df.replace({'평점':'324'},평점324);
chicken_df = chicken_df.replace({'평점':'325'},평점325);

chicken_df = chicken_df.replace({'평점':'331'},평점331); chicken_df = chicken_df.replace({'평점':'332'},평점332);
chicken_df = chicken_df.replace({'평점':'333'},평점333); chicken_df = chicken_df.replace({'평점':'334'},평점334);
chicken_df = chicken_df.replace({'평점':'335'},평점335);

chicken_df = chicken_df.replace({'평점':'341'},평점341); chicken_df = chicken_df.replace({'평점':'342'},평점342);
chicken_df = chicken_df.replace({'평점':'343'},평점343); chicken_df = chicken_df.replace({'평점':'344'},평점344);
chicken_df = chicken_df.replace({'평점':'345'},평점345);

chicken_df = chicken_df.replace({'평점':'351'},평점351); chicken_df = chicken_df.replace({'평점':'352'},평점352);
chicken_df = chicken_df.replace({'평점':'353'},평점353); chicken_df = chicken_df.replace({'평점':'354'},평점354);
chicken_df = chicken_df.replace({'평점':'355'},평점355);

### 평점 411~455 ###

chicken_df = chicken_df.replace({'평점':'411'},평점411); chicken_df = chicken_df.replace({'평점':'412'},평점412);
chicken_df = chicken_df.replace({'평점':'413'},평점413); chicken_df = chicken_df.replace({'평점':'414'},평점414);
chicken_df = chicken_df.replace({'평점':'415'},평점415);

chicken_df = chicken_df.replace({'평점':'421'},평점421); chicken_df = chicken_df.replace({'평점':'422'},평점422);
chicken_df = chicken_df.replace({'평점':'423'},평점423); chicken_df = chicken_df.replace({'평점':'424'},평점424);
chicken_df = chicken_df.replace({'평점':'425'},평점425);

chicken_df = chicken_df.replace({'평점':'431'},평점431); chicken_df = chicken_df.replace({'평점':'432'},평점432);
chicken_df = chicken_df.replace({'평점':'433'},평점433); chicken_df = chicken_df.replace({'평점':'434'},평점434);
chicken_df = chicken_df.replace({'평점':'435'},평점435);

chicken_df = chicken_df.replace({'평점':'441'},평점441); chicken_df = chicken_df.replace({'평점':'442'},평점442);
chicken_df = chicken_df.replace({'평점':'443'},평점443); chicken_df = chicken_df.replace({'평점':'444'},평점444);
chicken_df = chicken_df.replace({'평점':'445'},평점445);

chicken_df = chicken_df.replace({'평점':'451'},평점451); chicken_df = chicken_df.replace({'평점':'452'},평점452);
chicken_df = chicken_df.replace({'평점':'453'},평점453); chicken_df = chicken_df.replace({'평점':'454'},평점454);
chicken_df = chicken_df.replace({'평점':'455'},평점455);

### 평점 511~555 ###

chicken_df = chicken_df.replace({'평점':'511'},평점511); chicken_df = chicken_df.replace({'평점':'512'},평점512);
chicken_df = chicken_df.replace({'평점':'513'},평점513); chicken_df = chicken_df.replace({'평점':'514'},평점514);
chicken_df = chicken_df.replace({'평점':'515'},평점545);

chicken_df = chicken_df.replace({'평점':'521'},평점511); chicken_df = chicken_df.replace({'평점':'522'},평점512);
chicken_df = chicken_df.replace({'평점':'523'},평점513); chicken_df = chicken_df.replace({'평점':'524'},평점514);
chicken_df = chicken_df.replace({'평점':'525'},평점545);

chicken_df = chicken_df.replace({'평점':'531'},평점511); chicken_df = chicken_df.replace({'평점':'532'},평점512);
chicken_df = chicken_df.replace({'평점':'533'},평점513); chicken_df = chicken_df.replace({'평점':'534'},평점514);
chicken_df = chicken_df.replace({'평점':'535'},평점545);

chicken_df = chicken_df.replace({'평점':'541'},평점511); chicken_df = chicken_df.replace({'평점':'542'},평점512);
chicken_df = chicken_df.replace({'평점':'543'},평점513); chicken_df = chicken_df.replace({'평점':'544'},평점514);
chicken_df = chicken_df.replace({'평점':'545'},평점545);

chicken_df = chicken_df.replace({'평점':'551'},평점511); chicken_df = chicken_df.replace({'평점':'552'},평점512);
chicken_df = chicken_df.replace({'평점':'553'},평점513); chicken_df = chicken_df.replace({'평점':'554'},평점514);
chicken_df = chicken_df.replace({'평점':'555'},평점545);


In [7]:
chicken_df.tail()

,,가게명,총평점,주소,아이디,평점,날짜,리뷰
107546,105358,후라이드참잘하는집-화곡점,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,qa**님,4.666667,2022년 02월 15일,순삭.....ㅎㅎ 죄송합니다 넘 후딱 먹어서 사진 찍을 여유가...^^ 정말 맛있게...
107547,105359,후라이드참잘하는집-화곡점,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,oi**님,4.666667,2022년 02월 15일,사진을 깜빡하고 먹었어요ㅠㅜ 맛은 맛있습니당
107548,105360,후라이드참잘하는집-화곡점,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,wj**님,4.666667,2022년 02월 15일,맛도짱 서비스도 짱짱 또 시켜먹을게요~ 이벤트 많이 해주세요
107549,105361,후라이드참잘하는집-화곡점,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,dk**님,4.666667,2022년 02월 14일,배달도 예정보다 빠르게오고 항상 맛있습니다!!
107550,105362,후라이드참잘하는집-화곡점,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,qk**님,4.666667,2022년 02월 14일,역시 맛있네요 배달도 생각보다 빠르구요!!


In [33]:
# 평점 float 타입을 int 타입으로 반올림

chicken_df['평점'] = chicken_df['평점'].round(0).astype(int)

In [141]:
chicken_df = pd.read_csv('D:/pythonTest/DataShipJo/chicken_neg_im.csv', names=['가게명', '주소', '총평점', '아이디', '평점', '날짜', '리뷰'])
chicken_df.tail()

,가게명,주소,총평점,아이디,평점,날짜,리뷰
38244,또래오래-마포망원점,서울 마포구 망원동 414-5 1층,4.9,eh**님,4.0,2022년 01월 01일,너무 배고파서 급하게 먹느라 재대로 찍은 사진이 없네요 맛있게 잘 먹었습니다.
38245,또래오래-마포망원점,서울 마포구 망원동 414-5 1층,4.9,eh**님,4.0,2021년 12월 30일,"후라이드반 양념반으로 치킨은 늘 먹었지만, 로제떡볶이는 처음 주문해봤어요. 로제라도..."
38246,또래오래-마포망원점,서울 마포구 망원동 414-5 1층,4.9,eh**님,4.0,2021년 03월 29일,맛있게 먹었어요
38247,또래오래-마포망원점,서울 마포구 망원동 414-5 1층,4.9,eh**님,4.0,2020년 11월 26일,또래오래 처음 주문해봤는데 너무 맛있네용ㅠㅠ 그냥 양념은 너무 달다는 얘기가 있어서...
38248,또래오래-마포망원점,서울 마포구 망원동 414-5 1층,4.9,eh**님,4.0,2020년 10월 10일,곤듀치킨은 언제 먹어도 맛있네요ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ 서비스로 주신 맵떡도 맛있었어요~


In [144]:
chicken_df.isna().sum()

가게명    0
주소     0
총평점    0
아이디    0
평점     0
날짜     0
리뷰     0
dtype: int64

In [143]:
chicken_df.dropna(inplace=True)

In [34]:
chicken_df

,가게명,주소,총평점,아이디,평점,날짜,리뷰
1,BHC-시립대점,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,4.8,pr**님,3,4시간 전,양이 많이 줄어든 느낌 이네요.
2,BHC-시립대점,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,4.8,rh**님,3,20시간 전,혹시 hite 맥주 잘못 주신 것 아니에요? 생맥주라면 이런 맛이 안나오는데 ㅠㅠㅠ...
3,BHC-시립대점,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,4.8,mo**님,3,어제,사진 찍고싶었는데 까먹고 다 먹어버�f네용 담에 또 시킬게요!!!!
4,BHC-시립대점,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,4.8,ky**님,3,어제,냄새에 홀려 정신나간듯 먹다가 사진찍었네요ㅜㅜ 이케큰콜라가 서비스라니 번창하세요~~
5,BHC-시립대점,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,4.8,wi**님,3,2일 전,여긴 맛이 검증된곳이라.할말 없음
...,...,...,...,...,...,...,...
38244,또래오래-마포망원점,서울 마포구 망원동 414-5 1층,4.9,eh**님,4,2022년 01월 01일,너무 배고파서 급하게 먹느라 재대로 찍은 사진이 없네요 맛있게 잘 먹었습니다.
38245,또래오래-마포망원점,서울 마포구 망원동 414-5 1층,4.9,eh**님,4,2021년 12월 30일,"후라이드반 양념반으로 치킨은 늘 먹었지만, 로제떡볶이는 처음 주문해봤어요. 로제라도..."
38246,또래오래-마포망원점,서울 마포구 망원동 414-5 1층,4.9,eh**님,4,2021년 03월 29일,맛있게 먹었어요
38247,또래오래-마포망원점,서울 마포구 망원동 414-5 1층,4.9,eh**님,4,2020년 11월 26일,또래오래 처음 주문해봤는데 너무 맛있네용ㅠㅠ 그냥 양념은 너무 달다는 얘기가 있어서...


In [25]:
# 리뷰 결측값을 'N' 으로 대체
# 평점 결측값을 '0' 으로 대체

chicken_df['시간'].fillna('N', inplace=True)
chicken_df['리뷰'].fillna('N', inplace=True)
chicken_df['평점'].fillna(0,inplace=True)

In [130]:
df = pd.read_csv('D:/pythonTest/DataShipJo/chicken_train.csv', names=['가게명','총평점','주소','아이디','평점','날짜','리뷰'])

In [139]:
df = df.sample(20000)

In [77]:
# df.drop(['번호'], axis=1, inplace=True)

In [107]:
df.reset_index(drop=True)

,가게명,총평점,주소,아이디,평점,날짜,리뷰
0,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,de**님,1,2시간 전,잘 먹었습니다
1,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,pr**님,5,4시간 전,양이 많이 줄어든 느낌 이네요.
2,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,sk**님,5,8시간 전,오늘도 맛나게 잘 먹었습니다~~~
3,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,gh**님,5,10시간 전,맛있어요 양도 많아요
4,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,ki**님,5,17시간 전,맛있게 잘먹었어요 다음에 또주문할께요
...,...,...,...,...,...,...,...
198983,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,qa**님,5,2022년 02월 15일,순삭.....ㅎㅎ 죄송합니다 넘 후딱 먹어서 사진 찍을 여유가...^^ 정말 맛있게...,NaN
198984,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,oi**님,5,2022년 02월 15일,사진을 깜빡하고 먹었어요ㅠㅜ 맛은 맛있습니당,NaN
198985,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,wj**님,5,2022년 02월 15일,맛도짱 서비스도 짱짱 또 시켜먹을게요~ 이벤트 많이 해주세요,NaN
198986,4.7,서울특별시 강서구 화곡동 773-26 773-26번지 1층,dk**님,5,2022년 02월 14일,배달도 예정보다 빠르게오고 항상 맛있습니다!!,NaN


In [145]:
chicken_df.to_csv('chicken_neg_test_.csv', mode='a', header=False, encoding='utf-8-sig')